In [ ]:
import os
import re
import sys
import glob
import json
import shutil
import skimage
import imageio
import tifffile
import datetime
import requests
import jsonschema
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import ndimage
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 1

sys.path.append('/Users/keith.cheveralls/projects/opencell/')
from opencell.imaging.images import RawPipelineTIFF

sys.path.append('/Users/keith.cheveralls/projects/dragonfly-automation/')
import dragonfly_automation.utils

%aimport dragonfly_automation.qc.pipeline_plate_qc
from dragonfly_automation.fov_models import PipelineFOVScorer
from dragonfly_automation.qc.pipeline_plate_qc import PipelinePlateQC

In [ ]:
# resize images for the SOP
jpgs = glob.glob('/Users/keith.cheveralls/Box/KCC-slides/automated-pipeline-microscopy-SOP/*.JPG')
for jpg in jpgs:
    continue
    im = imageio.imread(jpg)
    imageio.imsave(jpg.replace('.JPG', '_RESZ4.JPG'), skimage.transform.downscale_local_mean(im, (4, 4, 1)))

In [ ]:
# get target names from API
result = requests.get('http://localhost:5000/lines?plate_id=P0019')
pd.DataFrame(data=result.json())[['cell_line_id', 'plate_id', 'well_id', 'target_name']].sort_values(by='well_id')

In [ ]:
# local acquisition directories (logs only)
pml_dirs = glob.glob('/Users/keith.cheveralls/image-data/raw-pipeline-microscopy/PML*/')
pml_dirs = sorted(pml_dirs)
len(pml_dirs)

In [ ]:
# real acquisitions (on ESS)
pml_dirs = glob.glob('/Volumes/ml_group/raw-pipeline-microscopy/PML*')
pml_dirs = sorted([pml_dir for pml_dir in pml_dirs if os.path.isdir(pml_dir)])
len(pml_dirs)

In [ ]:
# discarded acquisitions (on ESS)
pml_dirs = glob.glob('/Volumes/ml_group/raw-pipeline-microscopy/_PML*')
pml_dirs = sorted([pml_dir for pml_dir in pml_dirs if os.path.isdir(pml_dir)])
len(pml_dirs)

### Create the external_metadata files for existing experiments

In [ ]:
# load the google sheet export
google_sheet = pd.read_csv('/Users/keith.cheveralls/Downloads/2020-01-17-Pipeline-microscopy-master-key-PMLs.csv')
google_sheet.rename(columns={'id': 'pml_id'}, inplace=True)
google_sheet

In [ ]:
# generate the metadata JSON files from the google sheet
# (note that these files are created when the acquisition is initialized; 
# they were regenerated to eliminate variation in e.g., parental_line_id)

# the metadata attributes for canonical half-plate or full-plate imaging
columns = ['pml_id', 'parental_line', 'plate_id', 'platemap_type', 'imaging_round_id']

overwrite = False
for path in pml_dirs:
    
    # the directory name is exactly the pml_id
    pml_id = path.split(os.sep)[-1]
    
    # retrieve the entry in the google sheet for this pml_id
    if pml_id not in google_sheet.pml_id.values:
        print('pml_id %s not found in google sheet' % pml_id)
        continue
    metadata = google_sheet.loc[google_sheet.pml_id==pml_id].iloc[0][columns]
   
    # if there is a custom platemap, the metadata should contain only the pml_id and the platemap_type
    if metadata['platemap_type'] == 'custom':
        metadata = metadata[['pml_id', 'platemap_type']]

    dst_filepath = os.path.join(path, 'metadata.json')
    if os.path.isfile(dst_filepath) and not overwrite:
        print('metadata.json already exists in %s' % path)
    else:
        with open(dst_filepath, 'w') as file:
            json.dump(dict(metadata), file)

In [ ]:
# update all of the custom platemaps so that the parental_line is always consistent
overwrite = False
for dirpath in pml_dirs:
    filepaths = glob.glob(os.path.join(dirpath, '*platemap.csv'))
    if not filepaths:
        continue
    if len(filepaths) > 1:
        print('Warning: more than one platemap in %s' % dirpath)
    filepath = filepaths[0]
    print(filepath)

    platemap = pd.read_csv(filepath)
    platemap['parental_line'] = 'czML0383'
    if 'electroporation_id' in platemap.columns:
        platemap.drop('electroporation_id', axis=1, inplace=True)
        
    if overwrite:
        platemap.to_csv(filepath, index=False)

### Load all of the existing external metadata files

In [ ]:
rows = []
for path in pml_dirs:
    dirname = path.split(os.sep)[-1]
    filepath = glob.glob(os.path.join(path, 'metadata.json'))
    if len(filepath) != 1:
        print('More than one metadata file in %s' % path)

    with open(filepath.pop()) as file:
        metadata = json.load(file)
        if metadata.get('pml_id') is not None and metadata['pml_id'] != dirname:
            print('Warning')
            
        row = {'dirname': dirname}
        row.update(metadata)
        rows.append(row)

In [ ]:
all_md = pd.DataFrame(data=rows)
all_md.head()

### Concat all raw TIFF metadata

In [ ]:
mds = []
for pml_dir in pml_dirs:
    print(pml_dir)
    qc = PipelinePlateQC(pml_dir)
    try:
        md = qc.construct_fov_metadata(overwrite=True)
    except:
        md = qc.construct_fov_metadata(overwrite=True, renamed=True)
    mds.append(md)

In [ ]:
all_md = pd.concat(mds, axis=0, sort=False)
all_md.shape

In [ ]:
# rows with any NANs
all_md.loc[all_md.isna().sum(axis=1) > 0]

In [ ]:
# unique values
cols = ('parental_line', 'imaging_round_id', 'platemap_type', 'plate_id', 'pml_id', 'site_num')
for col in cols:
    print('%s: %s' % (col, np.array(sorted(all_md[col].unique()))))

In [ ]:
# fix existing metadata filenames
for pml_dir in pml_dirs:
    print(pml_dir)
    # os.rename(os.path.join(pml_dir, 'raw-tiff-metadata.csv'), os.path.join(pml_dir, 'fov-metadata.csv'))

In [ ]:
# test construct_fov_metadata after files have been renamed
qc = PipelinePlateQC('/Users/keith.cheveralls/image-data/raw-pipeline-microscopy/PML0227/')
md = qc.construct_fov_metadata(renamed=True, overwrite=True)

### Debugging

In [ ]:
qc = PipelinePlateQC('/Volumes/ml_group/raw-pipeline-microscopy/PML0244/')

### Parse the events log to check for lagging-snap issue

In [ ]:
log_filepaths = {}
for pml_dir in pml_dirs:
    qc = PipelinePlateQC(pml_dir)
    log_filepath = glob.glob(os.path.join(qc.log_dir, '*important-events.log'))[0]
    log_filepaths[qc.root_dirname] = log_filepath

In [ ]:
well_id = 'B2'
results = {}
for pml_id, filepath in log_filepaths.items():
    with open(filepath, 'r') as file:
        lines = file.readlines()
        inds = [ind for ind, line in enumerate(lines) if re.findall('Imaging \d FOVs in well %s' % well_id, line)]
        if not inds:
            results[pml_id] = None
            continue
        for line in lines[inds[0]:]:
            result = re.findall('max_intensity = (\d+) at z = (-?\d+)', line)
            if result:
                results[pml_id] = line
                break

In [ ]:
results